In [3]:
from scipy.integrate import solve_ivp

def overflow_event(t, y):
    return 1E9 - max(abs(yi) for yi in y)

# Define your ODE system
def ode_system(t, y):
    # Example system
    dydt = [-0.5*y[0], 4.5-y[1]**2]
    return dydt

# Initial conditions and time span
y0 = [1.0, 1.0]
t_span = (0, 10)

# Configure the event function
overflow_event.terminal = True  # Stop integration when event occurs
overflow_event.direction = -1   # Event occurs when function decreases through zero

# Solve the ODE system
solution = solve_ivp(ode_system, t_span, y0, events=overflow_event)

# Check the solution
if solution.status == 1:
    print("Integration stopped due to overflow event.")
else:
    print("Integration completed without overflow.")


Integration completed without overflow.


In [3]:
from scipy.integrate import solve_ivp
import numpy as np

# Define the ODE system
def exponential_decay(t, y, k):
    return -k * y

# Constants
k = 0.3

# Initial condition
y_initial = [10]

# Time span for the solution
t_span = (0, 10)

# Time points at which to store the computed solution
t_eval = np.linspace(t_span[0], t_span[1], 100)

# Solving the ODE
sol = solve_ivp(exponential_decay, t_span, y_initial, args=(k,), t_eval=t_eval, dense_output=True)

# Printing the result
print("Time points (t):", np.shape(sol.t))
print("Solution (y):", np.shape(sol.y))
print("Dense Output (sol):", sol.sol)
print("Event Times (t_events):", sol.t_events)
print("Event Solutions (y_events):", sol.y_events)
print("Number of RHS evaluations (nfev):", sol.nfev)
print("Number of Jacobian evaluations (njev):", sol.njev)
print("Number of LU decompositions (nluint):", sol.nlu)
print("Termination status (status):", sol.status)
print("Termination message (message):", sol.message)
print("Success flag (success):", sol.success)


Time points (t): (100,)
Solution (y): (1, 100)
Dense Output (sol): <scipy.integrate._ivp.common.OdeSolution object at 0x000002246D746710>
Event Times (t_events): None
Event Solutions (y_events): None
Number of RHS evaluations (nfev): 32
Number of Jacobian evaluations (njev): 0
Number of LU decompositions (nluint): 0
Termination status (status): 0
Termination message (message): The solver successfully reached the end of the integration interval.
Success flag (success): True


In [5]:
import numpy as np
from scipy.integrate import solve_ivp
import time
import sys

# Define the ODE system (same exponential decay example)
def exponential_decay(t, y, k):
    return -k * y

# Constants
k = 0.3

# Initial condition and time span
y_initial = [10]
t_span = (0, 10)
t_eval = np.linspace(t_span[0], t_span[1], 100)

# Function to estimate the size of an object in bytes (previously defined)
def estimate_size(obj, seen=None):
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([estimate_size(v, seen) for v in obj.values()])
        size += sum([estimate_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += estimate_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([estimate_size(i, seen) for i in obj])
    return size

# Timing and size estimation function
def solve_and_estimate(dense_output):
    start_time = time.time()
    sol = solve_ivp(exponential_decay, t_span, y_initial, args=(k,), t_eval=t_eval, dense_output=dense_output)
    elapsed_time = time.time() - start_time
    
    sol_y_size = estimate_size(sol.y)
    sol_t_size = estimate_size(sol.t)
    
    # Assuming double precision floating point for disk size estimate
    bytes_per_value = 8
    disk_size_estimate_y = sol.y.size * bytes_per_value
    disk_size_estimate_t = sol.t.size * bytes_per_value
    
    if dense_output:
        dense_output_size = estimate_size(sol.sol)
        print("With Dense Output:")
        print("Dense Output Size in Memory:", dense_output_size, "bytes")
        # Estimating disk size for dense output requires knowledge of the interpolant's storage structure
        # Here we provide a rough approximation assuming it's similar to storing y and t
        disk_size_estimate_dense_output = dense_output_size # This is a simplification
        print("Estimated Dense Output Size on Disk:", disk_size_estimate_dense_output, "bytes")
    else:
        print("Without Dense Output:")
    
    print("Execution Time:", elapsed_time, "seconds")
    print("sol.y Size in Memory:", sol_y_size, "bytes")
    print("sol.t Size in Memory:", sol_t_size, "bytes")
    print("Estimated sol.y Size on Disk:", disk_size_estimate_y, "bytes")
    print("Estimated sol.t Size on Disk:", disk_size_estimate_t, "bytes")
    print("---")

# Solve and estimate without dense output
solve_and_estimate(dense_output=False)

# Solve and estimate with dense output
solve_and_estimate(dense_output=True)


Without Dense Output:
Execution Time: 0.0010006427764892578 seconds
sol.y Size in Memory: 4240 bytes
sol.t Size in Memory: 4112 bytes
Estimated sol.y Size on Disk: 800 bytes
Estimated sol.t Size on Disk: 800 bytes
---
With Dense Output:
Dense Output Size in Memory: 4857 bytes
Estimated Dense Output Size on Disk: 4857 bytes
Execution Time: 0.0009963512420654297 seconds
sol.y Size in Memory: 4240 bytes
sol.t Size in Memory: 4112 bytes
Estimated sol.y Size on Disk: 800 bytes
Estimated sol.t Size on Disk: 800 bytes
---


In [7]:
import numpy as np
from scipy.integrate import solve_ivp
import time

# Define the ODE system (exponential decay)
def exponential_decay(t, y, k):
    return -k * y

# Constants
k = 0.3  # Decay rate
y_initial = [10]  # Initial condition
t_span = (0, 10)  # Time span for the solution

# Define the methods to compare
methods = ['RK45', 'RK23', 'DOP853', 'Radau', 'BDF', 'LSODA']

# Iterate through each method and solve the ODE
for method in methods:
    start_time = time.time()
    sol = solve_ivp(exponential_decay, t_span, y_initial, method=method, args=(k,), dense_output=True)
    elapsed_time = time.time() - start_time
    
    # Print results for each method
    print(f"Method: {method}")
    print(f"Execution Time: {elapsed_time:.4f} seconds")
    print(f"Number of Function Evaluations: {sol.nfev}")
    print(f"Number of Jacobian Evaluations: {sol.njev}")
    print(f"Number of LU Decompositions: {sol.nlu}")
    print(f"Termination Status: {sol.status} ({sol.message})")
    print(f"Success: {sol.success}")
    print("---")


Method: RK45
Execution Time: 0.0010 seconds
Number of Function Evaluations: 32
Number of Jacobian Evaluations: 0
Number of LU Decompositions: 0
Termination Status: 0 (The solver successfully reached the end of the integration interval.)
Success: True
---
Method: RK23
Execution Time: 0.0020 seconds
Number of Function Evaluations: 32
Number of Jacobian Evaluations: 0
Number of LU Decompositions: 0
Termination Status: 0 (The solver successfully reached the end of the integration interval.)
Success: True
---
Method: DOP853
Execution Time: 0.0000 seconds
Number of Function Evaluations: 47
Number of Jacobian Evaluations: 0
Number of LU Decompositions: 0
Termination Status: 0 (The solver successfully reached the end of the integration interval.)
Success: True
---
Method: Radau
Execution Time: 0.0260 seconds
Number of Function Evaluations: 57
Number of Jacobian Evaluations: 2
Number of LU Decompositions: 8
Termination Status: 0 (The solver successfully reached the end of the integration interv

In [8]:
import numpy as np
from scipy.integrate import solve_ivp, odeint
import time

# Define the ODE system
def exponential_decay(t, y, k):
    return -k * y

# Constants and parameters
k = 0.3
y_initial = [10]
t_span = (0, 10)
t_eval = np.linspace(t_span[0], t_span[1], 100)

# Benchmark solve_ivp
start_time = time.time()
sol_ivp = solve_ivp(lambda t, y: exponential_decay(t, y, k), t_span, y_initial, method='RK45', t_eval=t_eval)
time_ivp = time.time() - start_time

# Benchmark odeint
start_time = time.time()
y_odeint = odeint(lambda y, t: exponential_decay(t, y, k), y_initial, t_eval)
time_odeint = time.time() - start_time

print(f"solve_ivp (RK45) Execution Time: {time_ivp:.4f} seconds")
print(f"odeint Execution Time: {time_odeint:.4f} seconds")


solve_ivp (RK45) Execution Time: 0.0010 seconds
odeint Execution Time: 0.0010 seconds
